In [1]:
%%javascript
require.config({ 
     paths: { 
     d3: 'https://d3js.org/d3.v7.min'
}});

require(["d3"], function(d3) {
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [3]:
import msprime
import random
from visualizer import visualizer


# Generate a random tree sequence with record_full_arg=True so that you get marked recombination nodes
rs = random.randint(0,10000)   
ts = msprime.sim_ancestry(
    samples=2,
    recombination_rate=1e-8,
    sequence_length=3_000,
    population_size=10_000,
    record_full_arg=True,
    random_seed=rs
)

#print(ts.tables)

print("random seed:", rs)
print(ts.draw_text())
d3arg = visualizer.D3ARG(ts=ts)
d3arg.draw(width=1500, height=500, y_axis_scale="time", y_axis_labels=True)

random seed: 4427
12753.26┊         ┊    12   ┊    12   ┊  
        ┊         ┊   ┏━┻━┓ ┊   ┏━┻━┓ ┊  
12498.36┊  11     ┊  11   ┃ ┊  11   ┃ ┊  
        ┊ ┏━┻━┓   ┊ ┏━┻┓  ┃ ┊   ┃   ┃ ┊  
7546.65 ┊ ┃  10   ┊ ┃ 10  ┃ ┊   ┃   ┃ ┊  
        ┊ ┃  ┏┻━┓ ┊ ┃  ┃  ┃ ┊   ┃   ┃ ┊  
5102.88 ┊ 9  ┃  ┃ ┊ 9  ┃  ┃ ┊   9   ┃ ┊  
        ┊ ┃  ┃  ┃ ┊ ┃  ┃  ┃ ┊ ┏━┻┓  ┃ ┊  
3126.88 ┊ ┃  7  ┃ ┊ ┃  7  ┃ ┊ ┃  8  ┃ ┊  
        ┊ ┃  ┃  ┃ ┊ ┃  ┃  ┃ ┊ ┃  ┃  ┃ ┊  
3064.86 ┊ ┃  6  ┃ ┊ ┃  6  ┃ ┊ ┃  6  ┃ ┊  
        ┊ ┃ ┏┻┓ ┃ ┊ ┃ ┏┻┓ ┃ ┊ ┃ ┏┻┓ ┃ ┊  
2965.53 ┊ ┃ ┃ ┃ 4 ┊ ┃ ┃ ┃ 5 ┊ ┃ ┃ ┃ 5 ┊  
        ┊ ┃ ┃ ┃ ┃ ┊ ┃ ┃ ┃ ┃ ┊ ┃ ┃ ┃ ┃ ┊  
0.00    ┊ 0 1 2 3 ┊ 0 1 2 3 ┊ 0 1 2 3 ┊  
        0        800      2780      3000 



In [3]:
import json
arg_json = json.load(open("example.json", "r"))
visualizer.draw_D3(arg_json=arg_json)

In [4]:
# Generate a random tree sequence with record_full_arg=True so that you get marked recombination nodes
rs = random.randint(0,10000)   
ts = msprime.sim_ancestry(
    samples=2,
    recombination_rate=1e-8,
    sequence_length=3_000,
    population_size=10_000,
    record_full_arg=True,
    random_seed=rs
)

#print(ts.tables)

print("random seed:", rs)
print(ts.draw_text())
d3arg = visualizer.D3ARG(ts=ts)
d3arg.draw(width=500, height=500, y_axis_scale="time", y_axis_labels=True)

random seed: 323
17786.90┊     6   ┊  
        ┊   ┏━┻━┓ ┊  
3587.99 ┊   5   ┃ ┊  
        ┊ ┏━┻┓  ┃ ┊  
3334.62 ┊ ┃  4  ┃ ┊  
        ┊ ┃ ┏┻┓ ┃ ┊  
0.00    ┊ 0 1 3 2 ┊  
        0       3000 

